# Example notebook

This notebook demonstrates the analysis and visualization of sound data that was done in order to model the impact of trucks on noise pollution in Red Hook.

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
import datetime 
from datetime import timedelta
import pytz
from pytz import timezone
from numpy import load
import os
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA as sklearnPCA
import h5py
import pylab
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import librosa
import matplotlib.dates as md
import sys

In [3]:
sys.path.insert(1, '../modules')
import data
import display

Loading the hdf5 files

In [4]:
h5 = h5py.File('../data/sound_data_improved.hdf5', 'r')

In [5]:
d = h5['sound_data']

# Creating subsample of 10000 points from all four sensors

In [6]:
sample_nums = np.random.choice(range(d.shape[0]), 10000, replace = False)

In [7]:
index = np.zeros(d.shape[0]).astype('bool')
index[sample_nums] = True

# Reading and graphing data from one sensor

Reading June data from one of the sensors

In [8]:
df = pd.read_csv('../data/sonycnode-b827ebc178d2.sonyc.csv', skiprows = 2, low_memory = False)
df.head()

,timestamp,dBAS
0,1.559362e+09,63.53
1,1.559362e+09,63.73
2,1.559362e+09,64.94
3,1.559362e+09,63.09
4,1.559362e+09,61.66


Creating two numpy array with the timestamps for June and the corresponding SPL values for June

In [9]:
time_arr = np.empty(df.shape[0], dtype = datetime.datetime)
timestamp_arr = df['timestamp'].values
dBAS_arr = df['dBAS'].values

In [10]:
time_arr = [data.convert_timestamps(x) for x in timestamp_arr]

Creating a new dataframe with timestamps in datetime format

In [11]:
time_df = df
time_df['timestamp'] = time_arr
time_df.tail()

,timestamp,dBAS
2589895,2019-07-01 00:00:38.810000-04:00,66.04
2589896,2019-07-01 00:00:39.810000-04:00,69.75
2589897,2019-07-01 00:00:40.810000-04:00,68.68
2589898,2019-07-01 00:00:41.810000-04:00,73.17
2589899,2019-07-01 00:00:42.810000-04:00,77.04


In [12]:
time_df.head()

,timestamp,dBAS
0,2019-06-01 00:00:42.690000-04:00,63.53
1,2019-06-01 00:00:43.690000-04:00,63.73
2,2019-06-01 00:00:44.690000-04:00,64.94
3,2019-06-01 00:00:45.690000-04:00,63.09
4,2019-06-01 00:00:46.690000-04:00,61.66


# Clustering on 45 dimensions

Running PCA on the embeddings in order to reduce dimensionality to 45

In [13]:
pca_45 = sklearnPCA(45)
projected_45 = pca_45.fit_transform(d['feature_vector'])

In [14]:
b827ebc178d2_mask = data.get_sensor_mask(d['sensor_id'], b'sonycnode-b827ebc178d2.sonyc')
b827ebc178d2_transformed = projected_45[b827ebc178d2_mask]
b827ebc178d2_timestamps = d[b827ebc178d2_mask, 'timestamp']
b827ebc178d2_timestamps_dt = [data.convert_timestamps(x) for x in b827ebc178d2_timestamps]

# Dataframe with timestamps and cluster assignment

In [15]:
truck_clusters = [5, 10, 11, 18, 20, 37, 42, 57, 63]

Dataframe spanning over the whole month


In [16]:
all_cluster_assignments = data.get_cluster_assignments(64, b827ebc178d2_transformed, projected_45[index])
seconds_b827ebc178d2_timestamps_dt = [x.replace(microsecond=0) for x in b827ebc178d2_timestamps_dt]

In [17]:
assignments_df = pd.DataFrame(data={'assignment':all_cluster_assignments}, \
                              index = seconds_b827ebc178d2_timestamps_dt)
assignments_df.head()

,assignment
2019-06-01 00:00:16-04:00,59
2019-06-01 00:00:17-04:00,54
2019-06-01 00:00:18-04:00,54
2019-06-01 00:00:19-04:00,10
2019-06-01 00:00:20-04:00,54


Removed all duplicate entries in dataframe

In [18]:
removed_assignments_df = assignments_df[~assignments_df.index.duplicated()]

In [19]:
complete = pd.date_range(datetime.datetime(2019, 6, 1, 4), datetime.datetime(2019, 7, 1, 4), periods=3600*24*30)
seconds_complete = [x.replace(microsecond=0, nanosecond=0) for x in complete]
aware_seconds_complete = [pytz.utc.localize(x) for x in seconds_complete]
tz_seconds_complete = [x.astimezone(pytz.timezone('US/Eastern')) for x in aware_seconds_complete]

Replaces actual cluster assignment with 0 for no assignment value, 1 for a truck cluster, 2 for every other cluster

In [20]:
complete_assignments_df = removed_assignments_df.reindex(tz_seconds_complete, axis='index', fill_value = 0)

In [21]:
complete_assignments_df = complete_assignments_df.replace(truck_clusters, 1)
complete_assignments_df = complete_assignments_df.replace(range(2,64), 2)

In [22]:
complete_assignments_df.head()

,assignment
2019-06-01 00:00:00-04:00,0
2019-06-01 00:00:01-04:00,0
2019-06-01 00:00:02-04:00,0
2019-06-01 00:00:03-04:00,0
2019-06-01 00:00:04-04:00,0


In [23]:
complete_assignments_df.tail()

,assignment
2019-06-30 23:59:55-04:00,2
2019-06-30 23:59:56-04:00,0
2019-06-30 23:59:57-04:00,0
2019-06-30 23:59:58-04:00,0
2019-07-01 00:00:00-04:00,0


# Dataframe with timestamp and SPL value

In [24]:
time_df.head()

,timestamp,dBAS
0,2019-06-01 00:00:42.690000-04:00,63.53
1,2019-06-01 00:00:43.690000-04:00,63.73
2,2019-06-01 00:00:44.690000-04:00,64.94
3,2019-06-01 00:00:45.690000-04:00,63.09
4,2019-06-01 00:00:46.690000-04:00,61.66


In [25]:
naive_time_df = [x.replace(tzinfo=None) for x in time_df['timestamp']]

In [26]:
seconds_complete_timestamp = [x.replace(microsecond=0) for x in time_df['timestamp']]

In [27]:
spl_df = pd.DataFrame(data={'dBAS': dBAS_arr}, index=seconds_complete_timestamp)

In [28]:
spl_df.tail()

,dBAS
2019-07-01 00:00:38-04:00,66.04
2019-07-01 00:00:39-04:00,69.75
2019-07-01 00:00:40-04:00,68.68
2019-07-01 00:00:41-04:00,73.17
2019-07-01 00:00:42-04:00,77.04


# Test Join

In [29]:
len(removed_assignments_df)

1422967

In [30]:
len(complete_assignments_df)

2592000

In [31]:
test_join_df = spl_df.join(removed_assignments_df, how='left')

In [32]:
test_join_df.head()

,dBAS,assignment
2019-06-01 00:00:42-04:00,63.53,27.0
2019-06-01 00:00:43-04:00,63.73,14.0
2019-06-01 00:00:44-04:00,64.94,14.0
2019-06-01 00:00:45-04:00,63.09,20.0
2019-06-01 00:00:46-04:00,61.66,14.0


In [33]:
test_join_df.tail()

,dBAS,assignment
2019-07-01 00:00:38-04:00,66.04,NaN
2019-07-01 00:00:39-04:00,69.75,NaN
2019-07-01 00:00:40-04:00,68.68,NaN
2019-07-01 00:00:41-04:00,73.17,NaN
2019-07-01 00:00:42-04:00,77.04,NaN


In [34]:
len(test_join_df)

2589900

In [35]:
test_join_df = test_join_df.replace(truck_clusters, 1)
test_join_df = test_join_df.replace(range(2,64), 2)

In [36]:
test_join_df.loc[pd.isnull(test_join_df['assignment']), 'assignment'] = 0

In [37]:
test_join_df.tail()

,dBAS,assignment
2019-07-01 00:00:38-04:00,66.04,0.0
2019-07-01 00:00:39-04:00,69.75,0.0
2019-07-01 00:00:40-04:00,68.68,0.0
2019-07-01 00:00:41-04:00,73.17,0.0
2019-07-01 00:00:42-04:00,77.04,0.0


# Joining cluster assignment dataframe and SPL dataframe

This dataframe is for the whole month, so if you slice it based on date, it will give you different subsets of information. There is no need to recreate any dataframes.

In [38]:
all_joined_df = spl_df.join(complete_assignments_df)

In [39]:
all_joined_df.loc[pd.isnull(all_joined_df['assignment']), 'assignment'] = 0

Gets rid of the extra 42 seconds of data in July 1st

In [40]:
all_joined_df_cut = all_joined_df[:-42]

In [41]:
all_joined_df_cut.head()

,dBAS,assignment
2019-06-01 00:00:42-04:00,63.53,2.0
2019-06-01 00:00:43-04:00,63.73,2.0
2019-06-01 00:00:44-04:00,64.94,2.0
2019-06-01 00:00:45-04:00,63.09,1.0
2019-06-01 00:00:46-04:00,61.66,2.0


In [42]:
all_joined_df_cut.tail()

,dBAS,assignment
2019-06-30 23:59:56-04:00,60.74,0.0
2019-06-30 23:59:57-04:00,59.08,0.0
2019-06-30 23:59:58-04:00,58.32,0.0
2019-06-30 23:59:59-04:00,59.30,0.0
2019-07-01 00:00:00-04:00,59.43,0.0


# Creating matrix of SPL values over each day

Matrix of time and day, containing SPL values for every second of every day (only weekdays)

In [43]:
spl_complete = spl_df['dBAS']

Indices just for the first 42 seconds

In [44]:
beginning_spl_indices = \
pd.date_range(datetime.datetime(2019, 6, 1, 4, 0, 0), datetime.datetime(2019, 6, 1, 4, 0, 42), periods=42)
beginning_spl_indices = [x.replace(microsecond=0, nanosecond=0) for x in beginning_spl_indices]
beginning_spl_indices = [pytz.utc.localize(x) for x in beginning_spl_indices]
beginning_spl_indices = [x.astimezone(pytz.timezone('US/Eastern')) for x in beginning_spl_indices]

In [45]:
beginning_spl = pd.Series(np.nan, index=beginning_spl_indices)

In [46]:
spl_complete_2 = pd.concat([beginning_spl, spl_complete])

In [47]:
spl_complete_month = spl_complete_2[:-43]

In [48]:
spl_complete_month.head()

2019-06-01 00:00:00-04:00   NaN
2019-06-01 00:00:01-04:00   NaN
2019-06-01 00:00:02-04:00   NaN
2019-06-01 00:00:03-04:00   NaN
2019-06-01 00:00:04-04:00   NaN
dtype: float64

In [49]:
spl_complete_month.tail()

2019-06-30 23:59:55-04:00    61.39
2019-06-30 23:59:56-04:00    60.74
2019-06-30 23:59:57-04:00    59.08
2019-06-30 23:59:58-04:00    58.32
2019-06-30 23:59:59-04:00    59.30
dtype: float64

# Creating median array for weekends

Creating arrays of SPL collected on weekdays and weekends

In [40]:
spl_complete_month_weekends = spl_complete_month[spl_complete_month.index.dayofweek >= 5]

In [41]:
spl_complete_month_weekdays = spl_complete_month[spl_complete_month.index.dayofweek < 5]

Removing duplicate times from both arrays

In [42]:
spl_complete_month_weekends = spl_complete_month_weekends[~spl_complete_month_weekends.index.duplicated()]

In [43]:
spl_complete_month_weekdays = spl_complete_month_weekdays[~spl_complete_month_weekdays.index.duplicated()]

Getting median values for weekends and weekdays

In [44]:
weekend_medians = data.get_median(spl_complete_month_weekends)

[ 1  2  8  9 15 16 22 23 29 30]
1: 86337
2: 86338
8: 86335
9: 86201
15: 86335
16: 86338
22: 86335
23: 86244
29: 86335
30: 86340


../modules/data.py:315: RuntimeWarning: invalid value encountered in less
  complete_day_arr[complete_day_arr<1] = np.nan


KeyboardInterrupt: 

In [ ]:
weekday_medians = data.get_median(spl_complete_month_weekdays)

Adding the first 42 seconds of June that didn't get included in the measured SPL data

In [53]:
beginning_spl_indices_series = pd.Series(data=beginning_spl_indices)

Creating datetime indices spanning the whole month, with the additional 42 secs in the beginning of June

In [56]:
medians_df_index = pd.concat([beginning_spl_indices_series, all_joined_df.reset_index()['index']])

Filling an array that spans the whole month, then filling it with median values for one day that are repeated

In [58]:
len(spl_complete)

2589900

In [60]:
month_weekday_median = np.empty(len(medians_df_index))

In [61]:
len(medians_df_index)

2589942

In [ ]:
for i in range(len(medians_df_index)):
    month_weekday_median[i] = weekday_medians[i%len(weekday_medians)]

# Making dataframes with median values for the month

Creating dataframe with weekday medians for whole month

In [ ]:
weekday_medians_df = pd.DataFrame({'median_dBAS':month_weekday_median}, index=medians_df_index)

In [ ]:
weekday_medians_df.tail()

In [ ]:
weekend_medians_df_indices = weekday_medians_df.loc[weekday_medians_df.index.dayofweek>=5].index

In [ ]:
weekend_medians_df_values = np.empty(len(weekend_medians_df_indices))

Replacing weekday values in weekend times with the weekend median SPL

In [ ]:
for x in range(len(weekend_medians_df_values)):
    weekend_medians_df_values[x] = weekend_medians[x % len(weekend_medians)]

In [ ]:
weekend_medians_df_values

Making a dataframe with weekend medians

In [ ]:
weekend_medians_df = pd.DataFrame({'median_dBAS':weekend_medians_df_values}, index=weekend_medians_df_indices)

In [ ]:
weekend_medians_df = weekend_medians_df[~weekend_medians_df.index.duplicated()]

In [ ]:
weekend_medians_df.tail()

# Replacing weekend values in dataframe with correct values

Merging dataframe with weekday median values and dataframe with weekend median values. The dataframe with weekday median values has indices for the whole month. Indices that are on the weekend will be replaced with the weekend median values.

In [ ]:
both_medians_df = weekday_medians_df.merge(weekend_medians_df, how='outer', left_index=True, right_index=True)

median_dBAS_x is the weekday median values, median_dBAS_y is the weekend median values. Since this is an outer join, many values in median_dBAS_y will be NaN.

In [ ]:
both_medians_df.head()

Creates a new column that replaces weekday median values that are in the weekend index with the weekend median values.

In [ ]:
both_medians_df['median_dBAS'] = \
both_medians_df['median_dBAS_x'].where(both_medians_df['median_dBAS_y'].isnull(), \
                                                                              both_medians_df['median_dBAS_y'])

Example: 

In [ ]:
both_medians_df.loc['2019-06-28 23:00:33-04:00']

Gets rid of the columns with weekday and weekend median values, since we have a column with the correct median values for both weekdays and weekends.

In [ ]:
both_medians_df = both_medians_df.drop(['median_dBAS_x', 'median_dBAS_y'], axis=1)

In [ ]:
both_medians_df.head()

# Joining weekday and weekend medians to dataframe

Joining the median dataframe to the dataframe with SPL and cluster assignment

In [ ]:
all_joined_df_cut_median = all_joined_df_cut.join(both_medians_df)

In [ ]:
all_joined_df_cut_median.tail()

In [ ]:
removed_all_joined_df_cut_median = all_joined_df_cut_median[~all_joined_df_cut_median.index.duplicated()]

Saving dataframe to csv for later use

In [ ]:
removed_all_joined_df_cut_median.to_csv('../output/june_2019_df.csv')